  Describe how did you handle the data and all licenses that were respected.

## Setup

In [ ]:
!pip install geopandas== 0.12
!pip install matplotlib==3.6
!pip install networkx==2.8
!pip install numpy==1.23
!pip install pandas==1.5
!pip install pyproj==3.4
!pip install Shapely==1.7
!pip install osmnx

In [ ]:
import warnings

In [ ]:
try:
    import geopy
except ModuleNotFoundError as e:
    %pip install geopy==2.2.0
    import geopy
if geopy.__version__ != "2.2.0":
    !pip install -U geopy==2.2.0
    import geopy

In [ ]:
#If you can't run osmnx
#!apt install libspatialindex-dev
try:
  import osmnx  as ox
except ModuleNotFoundError as e:
  !pip install osmnx
  import osmnx  as ox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
try:
  import geopandas as gpd
except ModuleNotFoundError as e:
  %pip install geopandas
  import geopandas as gpd

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
try:
  import pygeos
except ModuleNotFoundError as e:
  %pip install pygeos
  import pygeos

In [ ]:
%pip install igraph
import igraph

In [ ]:
#Always after geopandas!!!
try:
  import pyrosm
except ModuleNotFoundError as e:
  %pip install pyrosm
  import pyrosm

In [ ]:
import requests
import geopandas as gpd
import fiona
import urllib
import os
import zipfile, io
import urllib3
import ssl
import shutil
from matplotlib import pyplot as plt
import networkx as nx
from geopy.geocoders import Nominatim


Now for the functions required to interact with Istat using colab:

In [ ]:

class CustomHttpAdapter(requests.adapters.HTTPAdapter):
  def __init__(self, ssl_context=None, **kwargs):
    self.ssl_context = ssl_context
    super().__init__(**kwargs)

  def init_poolmanager(self, connections, maxsize, block=False):
    self.poolmanager = urllib3.poolmanager.PoolManager(num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def get_legacy_session():
  ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
  ctx.options |= 0x4  
  session = requests.session()
  session.mount('https://', CustomHttpAdapter(ctx))
  return session

def download_file(url):
# source: https://stackoverflow.com/a/33488338
  local_filename = url.split('/')[-1]
  with get_legacy_session() as s:
    r = s.get(url, stream=True)
    r.raw.decode_content = True
    with open(local_filename, 'wb') as f:
      shutil.copyfileobj(r.raw, f)
  return local_filename

In [ ]:
if not os.path.exists('Limiti01012022'):
    zip_file_url = 'https://www.istat.it/storage/cartografia/confini_amministrativi/non_generalizzati/Limiti01012022.zip'
    zip_file_name = "Limiti01012022.zip"
    
    try: 
      urllib.request.urlretrieve(zip_file_url ,zip_file_name) 
    
    except:
      zip_file_name = download_file(zip_file_url)

    z = zipfile.ZipFile(zip_file_name)
    # unzip the file
    z.extractall()

In [ ]:
os.listdir('Limiti01012022')
os.chdir('Limiti01012022')
os.chdir('Com01012022')

for root, dirs, files in os.walk("."):  
    for filename in files:
        print(filename)

Com01012022_WGS84.dbf
Com01012022_WGS84.prj
Com01012022_WGS84.shx
Com01012022_WGS84.shp


In [ ]:
cities=gpd.read_file('Com01012022_WGS84.shp')

## Start

I want to compute the distance between two cities, assuming one travel by car.
First, we set as input one location and we localize it, plotting it and finding the province it belongs to. I'm using simplemaps's list of italian cities, available here:
https://simplemaps.com/data/it-cities

For now I try only cities belonging to the same region, as I tried using the whole country but it would require many Gb of RAM and storing capacities.

In [ ]:
start = input("Starting address: ")
point_1 = gpd.tools.geocode(start, provider="arcgis")    

Starting address: Modena


Here, make sure that we are selecting an italian city!!

Nominatim is an API which allows us to know the province, region and country of any given location.

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(str(float(point_1.geometry.y))+","+ str(float(point_1.geometry.x)))

start_address = location.raw['address']
#We get the province.

if start_address['country'] != 'Italia':
  print('Restart! Not an italian city')
print('Provincia:', start_address['county'])

Provincia: Modena


In [ ]:
start_address

{'house_number': '2',
 'road': 'Via Emilia Est',
 'suburb': 'Musicisti',
 'city': 'Modena',
 'county': 'Modena',
 'ISO3166-2-lvl6': 'IT-MO',
 'state': 'Emilia-Romagna',
 'ISO3166-2-lvl4': 'IT-45',
 'postcode': '41124',
 'country': 'Italia',
 'country_code': 'it'}

In [ ]:
city_start = cities[cities['COMUNE'] == start_address['city']]
city_start.plot()
print('Starting City: ', start_address['city'])
plt.show()


Now, I take a local map of car roads in order to be able to find the shortest route between two cities; the next step will be to compute its cost.

In [ ]:
# ox.config(log_console=True)
# gdf = ox.geocode_to_gdf('Italy')
# gdf.plot()

In [ ]:
area = start_address['county']
#USE COUNTY FOR TRIAL RUNS, STATE FOR THE ACTUAL DEMO.
area

In [ ]:
G = ox.graph_from_place(area, network_type='drive') #Only need to be ran once.

In [ ]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
ox.save_graphml(G, "region_roads.graphml")

In [ ]:
G = ox.load_graphml("region_roads.graphml")

In [ ]:
point_nearest_start = ox.distance.nearest_nodes(G,Y=location[1][0], X=location[1][1])

## End

In [ ]:
end = input("Destination: ")
point_2 = gpd.tools.geocode(end, provider="arcgis")

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location2 = geolocator.reverse(str(float(point_2.geometry.y))+","+ str(float(point_2.geometry.x)))

end_address = location2.raw['address']

In [ ]:
location2.raw

In [ ]:
if start_address['county'] !=  end_address['county']:
  print('Not from the same province!')

In [ ]:
city_end = cities[cities['COMUNE'] == end_address['town']]
city_end.plot()
print('Arriving city: ', end_address['town'])
plt.show()

In [ ]:
point_nearest_end = ox.distance.nearest_nodes(G,Y=location2[1][0], X=location2[1][1])

## Travel

In [ ]:
distance = point_1.to_crs('epsg:32632').geometry.distance(point_2.geometry.to_crs('epsg:32632')).values[0] / 1000
distance = round(distance, 3)
print('Distance: ', distance, 'km')

Here I plot the shortest route by distance.

In [ ]:
shortest_path_distance = ox.shortest_path(G, point_nearest_start, point_nearest_end, weight='length')
travel_time = nx.shortest_path_length(G, point_nearest_start, point_nearest_end, weight='travel_time')
minutes = travel_time // 60 
seconds = round(travel_time % 60*0.6)

km_distance = round(nx.shortest_path_length(G, point_nearest_start, point_nearest_end, weight='length')/1000, 3)

In [ ]:
#print('Expected Travel time: ', minutes, 'minutes and ', seconds, 'seconds')
print('Distance: ', km_distance,'km')
fig, ax = ox.plot_graph_route(G, shortest_path_distance, route_color='green', route_linewidth=6, node_size=0)
plt.show()

Now, I want to compute the travel cost; assuming one goes by car, I searched online for answers and for this project I'm assuming an average consumption of 1 liter of fuel for each 15 km travelled and a fuel cost of 1.65 € per liter.

Highway assumes an average speed of 100 km/h, freeway 70 km/h.
The main drawback is that going through highways has a price, which I found to be around 0.09€/km on average (of course, the exact price will change based on which part of the country you are travelling in, especially on the Northern regions).

Sources:
https://www.autostrade.it/en/il-pedaggio/come-si-calcola-il-pedaggio

https://www.trend-online.com/risparmio/autostrade-piu-care-italia-quali-sono-km/

https://www.globalpetrolprices.com/Italy/gasoline_prices/

In [ ]:
h_minute_time = round(km_distance*60//100)
h_seconds_time = round((km_distance * 60) % 100*0.6)
h_time= str(h_minute_time) + ' minutes, ' + str(h_seconds_time) + ' seconds '

f_minute_time = round(km_distance*60//70)
f_seconds_time = round((km_distance * 60) % 65*0.6)
f_time= str(f_minute_time) + ' minutes, ' + str(f_seconds_time) + ' seconds '

price = round(1.65/15 * km_distance,2)
f_cost = price
h_cost = price + round(0.09*km_distance,2)

print('Car travel information:\nHighway price = ', h_cost, '€', '; Highway time =', h_time)
print('Freeway price = ', f_cost,'€ ; Freeway time = ', f_time)

Optional: create isochrones by price based on distance!!

Alternative: Pyrosm maps!!!
RICHIEDE COMUNQUE TROPPA RAM

In [ ]:
region_list = {'Piemonte': '01_Piemonte_poly.osm.pbf', "Valle d'Aosta":"02_Valle d'Aosta - Vallée d'Aoste_poly.osm.pbf", 
               'Lombardia': '03_Lombardia_poly.osm.pbf', 'Trentino-Alto Adige/Südtirol': '04_Trentino-Alto Adige-Südtirol_poly.osm.pbf',
               'Veneto': '05_Veneto_poly.osm.pbf', 'Friuli-Venezia Giulia': '06_Friuli-Venezia Giulia_poly.osm.pbf', 'Liguria':'07_Liguria_poly.osm.pbf',
               'Emilia-Romagna':'08_Emilia-Romagna_poly.osm.pbf', 'Toscana':'09_Toscana_poly.osm.pbf', 'Umbria':'10_Umbria_poly.osm.pbf',
               'Marche':'11_Marche_poly.osm.pbf', 'Lazio':'12_Lazio_poly.osm.pbf', 'Abruzzo':'13_Abruzzo_poly.osm.pbf', 'Molise':'14_Molise_poly.osm.pbf',
               'Campania':'15_Campania_poly.osm.pbf', 'Puglia':'16_Puglia_poly.osm.pbf', 'Basilicata':'17_Basilicata_poly.osm.pbf',
               'Calabria':'18_Calabria_poly.osm.pbf','Sicilia':'19_Sicilia_poly.osm.pbf', 'Sardegna':'20_Sardigna-Sardegna_poly.osm.pbf'}

In [ ]:
url_download_region_pbf = 'https://osmit-estratti-test.wmcloud.org/dati/poly/regioni/pbf/' + region_list[start_address['state']]
urllib.request.urlretrieve(url_download_region_pbf ,"region_osm.pbf")

osm = pyrosm.OSM("region_osm.pbf") 

In [ ]:
nodes, edges = osm.get_network(nodes=True, network_type="driving")

In [ ]:
G = osm.to_graph(nodes, edges, graph_type="networkx")
route = ox.shortest_path(G, point_nearest_start, point_nearest_end, weight='length')
fig, ax = ox.plot_graph_route(G, route, route_linewidth=6, node_size=0, bgcolor='k')
plt.show()

NameError: ignored

In [ ]:
trip_times = [30, 45, 60, 75, 90]

f_travel_speed = 65  #Freeway
h_travel_speed = 100 #highway

center_node = point_nearest_start

#Freeway: 
meters_per_minute = f_travel_speed * 1000 / 60  # km per hour to m per minute
for orig,dest, p, data in G_proj.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G_proj, center_node, radius=trip_time, distance="time")
    node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

crs_proj = ox.graph_to_gdfs(G_proj)[0].crs


In [ ]:
data = {'trip_time': sorted(trip_times, reverse=True), 'geometry': isochrone_polys}
isochrones = gpd.GeoDataFrame(data,crs=crs_proj)
print('Freeway Isochrones')
isochrones.explore(column='trip_time',cmap='Reds')


I do the same for highway.

In [ ]:
meters_per_minute = h_travel_speed * 1000 / 60  # km per hour to m per minute
for orig,dest, p, data in G_proj.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G_proj, center_node, radius=trip_time, distance="time")
    node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

crs_proj = ox.graph_to_gdfs(G_proj)[0].crs

In [ ]:
data = {'trip_time': sorted(trip_times, reverse=True), 'geometry': isochrone_polys}
isochrones = gpd.GeoDataFrame(data,crs=crs_proj)
isochrones.explore(column='trip_time',cmap='Greens')
print('Highway Isochrones')